In [ ]:
# torch.nn.functional中有很多功能，后续会常用。那么什么是用nn.Module，什么时候用nn.functional呢
# 一般情况下，如果 模型有可学习的参数，最好用nn.Module，其他情况用nn.functional更简单一些
# 
import gzip
import numpy as np

mnist = {
    'trainset':'d:/GitRepos/data/mnist/train-images-idx3-ubyte.gz',
    'trainlabel':'d:/GitRepos/data/mnist/train-labels-idx1-ubyte.gz',
    'testset':'d:/GitRepos/data/mnist/t10k-images-idx3-ubyte.gz',
    'testlabel':'d:/GitRepos/data/mnist/t10k-labels-idx1-ubyte.gz'
}

